In [1]:
## imports

import numpy as np
import pandas as pd

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
pd.set_option('display.max_columns', 100)
from lightgbm import LGBMClassifier
from datetime import datetime
from catboost import CatBoostClassifier, Pool, cv

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder
seed = 45
#seed =145
% matplotlib inline
plt.style.use('fivethirtyeight')

In [3]:
### read data with some categorty name changes to raw files
PATH='F:/AV/WNS'
train_csv = 'train_catboost.csv'
test_csv = 'test_catboost.csv'
submit_csv = 'sample_submission_M0L0uXE.csv'

### read train, test and submission files
train = pd.read_csv(f'{PATH}/{train_csv}')
test = pd.read_csv(f'{PATH}/{test_csv}')
submission = pd.read_csv(f'{PATH}/{submit_csv}')

print("Shape of {}:{} {}:{} {}:{}".format('train',train.shape,'test',test.shape,'submission',submission.shape))

Shape of train:(54808, 14) test:(23490, 13) submission:(23490, 2)


In [4]:
### inspect data

train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales_Marketing,region_7,Masters_above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales_Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales_Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [5]:
### class balance

train['is_promoted'].value_counts()

## so approx 10% of past employees have been promoted

0    50140
1     4668
Name: is_promoted, dtype: int64

In [6]:
### lets check if there is any repeat in employees

len(train['employee_id'].unique()) == train.shape[0]

### so all IDs are unique

True

In [7]:
## null values

null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

## so 2 columns have null values

education               2409
previous_year_rating    4124
dtype: int64

In [8]:
### inspect null value columns
a = train[(train.education.isnull())]
_ = train[(train.education.isnull() | train.previous_year_rating.isnull())]

print(a.shape,_.shape)

(2409, 14) (6148, 14)


In [9]:
set(a.index).intersection(set(_.index)) == set(a.index)

### so everywhere where education is not present prev year rating is also not present, but vice-versa is not true

True

In [10]:
_.head(10)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
10,29934,Technology,region_23,NaN,m,sourcing,1,30,NaN,1,0,0,77,0
21,33332,Operations,region_15,NaN,m,sourcing,1,41,4.0,11,0,0,57,0
23,71177,Procurement,region_5,Bachelor's,m,other,1,27,NaN,1,0,0,70,0
29,74759,Sales_Marketing,region_4,Bachelor's,m,sourcing,1,26,NaN,1,0,0,44,0
32,35465,Sales_Marketing,region_7,NaN,f,sourcing,1,24,1.0,2,0,0,48,0
43,17423,Sales_Marketing,region_2,NaN,m,other,3,24,2.0,2,0,0,48,0
56,45709,Sales_Marketing,region_31,Bachelor's,f,other,1,29,NaN,1,0,0,49,0
58,26599,Sales_Marketing,region_16,Bachelor's,m,other,2,27,NaN,1,1,0,47,0
62,9150,Analytics,region_22,Bachelor's,f,other,1,28,NaN,1,1,0,80,0
66,77981,Finance,region_22,Bachelor's,m,other,1,27,NaN,1,1,1,58,1


In [11]:
### check avg % of people promoted with NA in previous ye rating vs without NA

print(np.mean(_['is_promoted']),np.mean(train[~train.education.isnull()]['is_promoted']))

0.07091737150292778 0.08675738086604706


In [12]:
### check avg % of people promoted with NA in education vs without NA

print(np.mean(a['is_promoted']),np.mean(train[~train.education.isnull()]['is_promoted']))

0.0506434205064342 0.08675738086604706


In [13]:
## For education we will use unknown for all missing values and 9999 for prev year training

train['education'] = train.education.fillna('unknown')
train['previous_year_rating'] = train.previous_year_rating.fillna(9999)

test['education'] = test.education.fillna('unknown')
test['previous_year_rating'] = test.previous_year_rating.fillna(9999)

In [14]:
### merge dataframes for ease of processing
Y = train['is_promoted'].values
train.drop('is_promoted',inplace=True,axis=1)
train['train'] = 'train'
test['train'] = 'test'
merged = pd.concat([train,test])
merged.shape

(78298, 14)

In [15]:
cat_cols = [i for i in merged.columns if merged[i].dtypes == 'object']+['KPIs_met >80%','awards_won?','previous_year_rating']
cat_cols

['department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'train',
 'KPIs_met >80%',
 'awards_won?',
 'previous_year_rating']

In [16]:
cat_cols.remove('train')
print(cat_cols)
#cat_idx = [merged.columns.get_loc(c) for c in merged.columns if c in cat_cols]

['department', 'region', 'education', 'gender', 'recruitment_channel', 'KPIs_met >80%', 'awards_won?', 'previous_year_rating']


In [17]:
merged.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,train
0,65438,Sales_Marketing,region_7,Masters_above,f,sourcing,1,35,5.0,8,1,0,49,train
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,train
2,7513,Sales_Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,train
3,2542,Sales_Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,train
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,train


In [18]:
tr_cols = [i for i in merged.columns]
tr_cols.remove('employee_id')
tr  = merged[merged['train']=='train']
te = merged[merged['train']=='test']
tr.drop('train',axis=1,inplace=True)
te.drop('train',axis=1,inplace=True)

print(tr_cols)

### ###
tr_cols.remove('train')
X_train = tr[tr_cols]
Y_train = Y
X_test = te[tr_cols]

#cat_idx = [X_train.columns.get_loc(c) for c in X_train.columns if c in cat_cols]
cat_idx = [X_train.columns.get_loc(c) for c in cat_cols]

train_pool = Pool(X_train, Y_train, cat_features=cat_idx)

C:\Users\ADMIN\Anaconda3\envs\Pytorch_env\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


['department', 'region', 'education', 'gender', 'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service', 'KPIs_met >80%', 'awards_won?', 'avg_training_score', 'train']


In [19]:
X_train.iloc[:,cat_idx].head()

,department,region,education,gender,recruitment_channel,KPIs_met >80%,awards_won?,previous_year_rating
0,Sales_Marketing,region_7,Masters_above,f,sourcing,1,0,5.0
1,Operations,region_22,Bachelor's,m,other,0,0,5.0
2,Sales_Marketing,region_19,Bachelor's,m,sourcing,0,0,3.0
3,Sales_Marketing,region_23,Bachelor's,m,other,0,0,1.0
4,Technology,region_26,Bachelor's,m,other,0,0,3.0


In [20]:
X_train.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,Sales_Marketing,region_7,Masters_above,f,sourcing,1,35,5.0,8,1,0,49
1,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60
2,Sales_Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50
3,Sales_Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50
4,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73


In [21]:
cat_idx

[0, 1, 2, 3, 4, 9, 10, 7]

In [22]:
# def pre_process(df,cat_cols):
#     one_hot_encoded_training_predictors = pd.get_dummies(df[cat_cols])
#     df.drop(cat_cols,inplace=True,axis=1)
#     _ = pd.concat([df,one_hot_encoded_training_predictors],1)
#     new_tr, new_tst = _[_['train']=='train'],_[_['train']=='test']
#     new_tr.drop('train',inplace=True,axis=1)
#     new_tst.drop('train',inplace=True,axis=1)
#     return new_tr, new_tst

In [23]:
# train_OHE,test_OHE = pre_process(merged,cat_cols)

In [24]:
def kfold_lcatboost(train_df,test_df, target,num_folds, stratified = False, debug= False,modelname="catboost"):
    # Divide in training/validation and test data
    train_df = train_df
    test_df = test_df
    print("Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    gc.collect()

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['employee_id','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], target)):
        train_x, train_y = train_df[feats].iloc[train_idx], target[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], target[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = CatBoostClassifier(
            #nthread=4,
            #class_weights = [1,2],
            bagging_temperature = 0.85,
            iterations=5000,
            learning_rate=0.01,
            l2_leaf_reg = 1,
            depth=10,
            loss_function='Logloss',
            eval_metric='F1',
            silent=False)
        #print(train_x.head())
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            verbose= 100, early_stopping_rounds= 200,cat_features=cat_idx,use_best_model=True)

        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d F-score : %.6f' % (n_fold + 1, sklearn.metrics.f1_score(valid_y, (oof_preds[valid_idx]>0.3).astype(int))))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full f1 score %.6f' % sklearn.metrics.f1_score(target, (oof_preds>0.3).astype(int)))
    
    # Write submission file and plot feature importance
    if not debug:
        _ = datetime.now().strftime('%Y%m%d%H%M%S')
        Fname = 'F:/AV/WNS/submission/'+str(modelname)+'_'+str(_)+'.csv'
        submission['is_promoted'] = sub_preds
        submission[['employee_id', 'is_promoted']].to_csv(Fname, index= False)
        oof = pd.DataFrame(oof_preds)
        score = sklearn.metrics.f1_score(target, (oof_preds>0.3).astype(int))
        oof.columns = [modelname+'_'+str(round(score,4))]
        OOF_Fname = 'F:/AV/WNS/oof/'+str(modelname)+'_'+str(_)+'.csv'
        oof.to_csv(OOF_Fname,index=False)
    #display_importances(feature_importance_df)
    return 

In [25]:
import gc
oof = kfold_lcatboost(X_train,X_test, Y,num_folds=5, stratified = True, debug= False,modelname="catboost_5fld_baggingtemp0.85")

Train shape: (54808, 12), test shape: (23490, 12)
0:	learn: 0.1374502	test: 0.1374502	best: 0.1374502 (0)	test1: 0.1477833	total: 166ms	remaining: 13m 48s
100:	learn: 0.2474275	test: 0.2478373	best: 0.3049166 (1)	test1: 0.2846715	total: 24.4s	remaining: 19m 43s
200:	learn: 0.3982888	test: 0.3982888	best: 0.3982888 (200)	test1: 0.4057725	total: 59.2s	remaining: 23m 33s
300:	learn: 0.4469320	test: 0.4482187	best: 0.4482187 (290)	test1: 0.4544709	total: 1m 41s	remaining: 26m 32s
400:	learn: 0.4717405	test: 0.4722675	best: 0.4722675 (399)	test1: 0.4781906	total: 2m 23s	remaining: 27m 26s
500:	learn: 0.4858529	test: 0.4871847	best: 0.4871847 (496)	test1: 0.4928230	total: 3m 4s	remaining: 27m 34s
600:	learn: 0.4900262	test: 0.4922582	best: 0.4922582 (591)	test1: 0.4992076	total: 3m 43s	remaining: 27m 16s
700:	learn: 0.4932663	test: 0.4954865	best: 0.4954865 (700)	test1: 0.5082612	total: 4m 19s	remaining: 26m 29s
800:	learn: 0.4961893	test: 0.4968944	best: 0.4971955 (798)	test1: 0.5094340	tot

500:	learn: 0.4805353	test: 0.4790905	best: 0.4790905 (492)	test1: 0.5200946	total: 2m 39s	remaining: 23m 53s
600:	learn: 0.4877852	test: 0.4861588	best: 0.4861588 (599)	test1: 0.5212598	total: 3m 15s	remaining: 23m 48s
700:	learn: 0.4930625	test: 0.4920539	best: 0.4920539 (699)	test1: 0.5220126	total: 3m 50s	remaining: 23m 32s
800:	learn: 0.4956853	test: 0.4951846	best: 0.4953834 (788)	test1: 0.5220126	total: 4m 25s	remaining: 23m 10s
900:	learn: 0.4991994	test: 0.4966927	best: 0.4966927 (881)	test1: 0.5227630	total: 5m 1s	remaining: 22m 53s
1000:	learn: 0.5012000	test: 0.4980981	best: 0.4980981 (999)	test1: 0.5227630	total: 5m 40s	remaining: 22m 40s
1100:	learn: 0.5032954	test: 0.4992000	best: 0.4995001 (1097)	test1: 0.5250784	total: 6m 25s	remaining: 22m 46s
1200:	learn: 0.5097416	test: 0.5012982	best: 0.5012982 (1185)	test1: 0.5242567	total: 7m 22s	remaining: 23m 18s
1300:	learn: 0.5162312	test: 0.5059713	best: 0.5062687 (1299)	test1: 0.5245902	total: 8m 26s	remaining: 23m 59s
1400

3200:	learn: 0.5912162	test: 0.5446497	best: 0.5447546 (3148)	test1: 0.4868735	total: 30m 25s	remaining: 17m 6s
3300:	learn: 0.5939666	test: 0.5454196	best: 0.5455245 (3293)	test1: 0.4868735	total: 31m 35s	remaining: 16m 15s
3400:	learn: 0.5985429	test: 0.5465385	best: 0.5465385 (3379)	test1: 0.4868735	total: 32m 43s	remaining: 15m 23s
3500:	learn: 0.6010073	test: 0.5470973	best: 0.5472713 (3491)	test1: 0.4868735	total: 33m 52s	remaining: 14m 30s
3600:	learn: 0.6049107	test: 0.5469231	best: 0.5472713 (3491)	test1: 0.4868735	total: 34m 59s	remaining: 13m 35s
3700:	learn: 0.6085343	test: 0.5473765	best: 0.5477609 (3645)	test1: 0.4868735	total: 36m 8s	remaining: 12m 41s
3800:	learn: 0.6105965	test: 0.5490121	best: 0.5490121 (3782)	test1: 0.4880763	total: 37m 16s	remaining: 11m 45s
3900:	learn: 0.6129092	test: 0.5494632	best: 0.5498466 (3845)	test1: 0.4876886	total: 38m 23s	remaining: 10m 49s
4000:	learn: 0.6145602	test: 0.5504025	best: 0.5504025 (3974)	test1: 0.4876886	total: 39m 31s	rema